# Speech Separation by Pretrained Models

In [ ]:
%%bash
#git clone https://github.com/tky823/DNN-based_source_separation.git

cd "./DNN-based_source_separation/egs/tutorials/"

# To install torch & torchaudio
#pip install -r requirements.txt

In [ ]:
%%bash
# Download speech dataset
for spk in aew axb bdl ; do
    wget "http://festvox.org/cmu_arctic/packed/cmu_us_${spk}_arctic.tar.bz2"
    tar -xjvf "./cmu_us_${spk}_arctic.tar.bz2" 
done

In [ ]:
import sys
import os
import numpy as np

In [ ]:
sys.path.append("/content/DNNSSgit/DNN-based_source_separation/src")

In [ ]:
import IPython.display as ipd
import torch
import torchaudio

In [ ]:
torch.manual_seed(111)
print(os.getcwd())
#os.chdir("DNSSgit/DNN-based_source_separation/src")
os.chdir("DNNSS/src")
print(os.getcwd())

C:\users\raghu\test
C:\users\raghu\test\DNNSS\src


In [ ]:
from models.deep_clustering import DeepClustering
from models.danet import DANet, FixedAttractorDANet
from models.adanet import ADANet
from models.lstm_tasnet import LSTMTasNet
from models.conv_tasnet import ConvTasNet
from models.post_processing_tasnet import pick_strongest_signal
from models.dprnn_tasnet import DPRNNTasNet
from models.dptnet import DPTNet
from models.sepformer import SepFormer
from criterion.sdr import sisdr

In [ ]:
waveform_aew, sample_rate = torchaudio.load("C:/users/raghu/test/content/cmu_us_aew_arctic/wav/arctic_a0001.wav")
waveform_axb, sample_rate = torchaudio.load("C:/users/raghu/test/content/cmu_us_axb_arctic/wav/arctic_a0002.wav")
waveform_bdl, sample_rate = torchaudio.load("C:/users/raghu/test/content/cmu_us_bdl_arctic/wav/arctic_a0003.wav")
waveform_aew, sample_rate = torchaudio.load("C:/users/raghu/test/test_audio_11.wav")
#waveform_axb, sample_rate = torchaudio.load("C:/users/raghu/test/test_audio_12.wav")
#waveform_bdl, sample_rate = torchaudio.load("C:/users/raghu/test/test_audio_12.wav")

print(waveform_aew.size())
print(waveform_axb.size())
print(waveform_bdl.size())
print('sample rate of wav', sample_rate)
SAMPLE_RATE_WSJ0 = 16000

torch.Size([1, 95232])
torch.Size([1, 56960])
torch.Size([1, 53841])
sample rate of wav 16000


In [ ]:
resampler = torchaudio.transforms.Resample(sample_rate, SAMPLE_RATE_WSJ0)
waveform_aew = resampler(waveform_aew)
waveform_axb = resampler(waveform_axb)
waveform_bdl = resampler(waveform_bdl)

In [ ]:
T_min = min(waveform_aew.size(-1), waveform_axb.size(-1), waveform_bdl.size(-1))
waveform_aew, waveform_axb, waveform_bdl = waveform_aew[:, :T_min], waveform_axb[:, :T_min], waveform_bdl[:, :T_min]
display(ipd.Audio(waveform_aew, rate=SAMPLE_RATE_WSJ0))
display(ipd.Audio(waveform_axb, rate=SAMPLE_RATE_WSJ0))
display(ipd.Audio(waveform_bdl, rate=SAMPLE_RATE_WSJ0))

## 2 speakers

In [ ]:
n_sources = 2
mixture = waveform_aew + 3*waveform_bdl
display(ipd.Audio(mixture, rate=SAMPLE_RATE_WSJ0))

In [ ]:
model = DeepClustering.build_from_pretrained(task="wsj0-mix", sample_rate=SAMPLE_RATE_WSJ0, n_sources=n_sources)
wrapper_model = DeepClustering.TimeDomainWrapper(model, n_fft=model.n_fft, hop_length=model.hop_length, window_fn=model.window_fn)
wrapper_model.eval()

input = mixture.unsqueeze(dim=0)
with torch.no_grad():
    output = wrapper_model(input, threshold=model.threshold, n_sources=n_sources, iter_clustering=None)

output = output.squeeze(dim=0)
estimated = torch.split(output, [1]*n_sources, dim=0)

for idx in range(n_sources):
    display(ipd.Audio(estimated[idx], rate=SAMPLE_RATE_WSJ0))

In [ ]:
model = DANet.build_from_pretrained(task="wsj0-mix", sample_rate=SAMPLE_RATE_WSJ0, n_sources=n_sources)
wrapper_model = DANet.TimeDomainWrapper(model, n_fft=model.n_fft, hop_length=model.hop_length, window_fn=model.window_fn)
wrapper_model.eval()

input = mixture.unsqueeze(dim=0)
with torch.no_grad():
    output = wrapper_model(input, threshold=model.threshold, n_sources=n_sources, iter_clustering=None)

output = output.squeeze(dim=0)
estimated = torch.split(output, [1]*n_sources, dim=0)

for idx in range(n_sources):
    display(ipd.Audio(estimated[idx], rate=SAMPLE_RATE_WSJ0))

In [ ]:
model = FixedAttractorDANet.build_from_pretrained(task="wsj0-mix", sample_rate=SAMPLE_RATE_WSJ0, n_sources=n_sources)
wrapper_model = FixedAttractorDANet.TimeDomainWrapper(model, n_fft=model.n_fft, hop_length=model.hop_length, window_fn=model.window_fn)
wrapper_model.eval()

input = mixture.unsqueeze(dim=0)
with torch.no_grad():
    output = wrapper_model(input)

output = output.squeeze(dim=0)
estimated = torch.split(output, [1]*n_sources, dim=0)

for idx in range(n_sources):
    display(ipd.Audio(estimated[idx], rate=SAMPLE_RATE_WSJ0))

In [ ]:
model = ADANet.build_from_pretrained(task="wsj0-mix", sample_rate=SAMPLE_RATE_WSJ0, n_sources=n_sources)
wrapper_model = ADANet.TimeDomainWrapper(model, n_fft=model.n_fft, hop_length=model.hop_length, window_fn=model.window_fn)
wrapper_model.eval()

input = mixture.unsqueeze(dim=0)
with torch.no_grad():
    output = wrapper_model(input, threshold=40, n_sources=n_sources)

output = output.squeeze(dim=0)
estimated = torch.split(output, [1]*n_sources, dim=0)

for idx in range(n_sources):
    display(ipd.Audio(estimated[idx], rate=SAMPLE_RATE_WSJ0))

In [ ]:
model = LSTMTasNet.build_from_pretrained(task="wsj0-mix", sample_rate=SAMPLE_RATE_WSJ0, n_sources=n_sources)
model.eval()

input = mixture.unsqueeze(dim=0)
with torch.no_grad():
    output = model(input)

output = output.squeeze(dim=0)
estimated = torch.split(output, [1]*n_sources, dim=0)

for idx in range(n_sources):
    display(ipd.Audio(estimated[idx], rate=SAMPLE_RATE_WSJ0))

In [ ]:
def pick_strongest_signal(x1,x2):
    print("In pick")
  #  print(torch.norm(x1[1,:8000]), torch.norm(x2[1,:8000]))
    print('First one second')
    print(torch.norm(x1[0][:16000]), torch.norm(x2[0][:16000]))
    print('Entire signal')
    print(torch.norm(x1[0]), torch.norm(x2[0]))
    if torch.norm(x1[0][:16000]) > torch.norm(x1[0][:16000]):
        return x1
    return x2
    

In [ ]:
#model = ConvTasNet.build_from_pretrained(task="wsj0-mix", sample_rate=SAMPLE_RATE_WSJ0, n_sources=n_sources)
model = ConvTasNet.build_from_pretrained(task="librispeech", sample_rate=16000, n_sources=n_sources)
print(model.get_config())
model.eval()
print(mixture.size())
input = mixture.unsqueeze(dim=0)
print('Input size', input.size())
chunk_size=8192
pad_size=128

with torch.no_grad():
    output = model(input)

    print(output.size())
    

#output = final_output.squeeze(dim=0)
output = output.squeeze(dim=0)
estimated = torch.split(output, [1]*n_sources, dim=0)
print('Calling pick strongest signal')
strong_signal = pick_strongest_signal(estimated[0], estimated[1])
#print(estimated[0].size())
#print(strong_signal.size())
#print(output.size(), waveform_axb.size())
#print(sisdr(estimated[0],waveform_axb ))
#print(sisdr(estimated[1],waveform_aew ))




for idx in range(n_sources):
    display(ipd.Audio(estimated[idx], rate=SAMPLE_RATE_WSJ0))
print("the strong signal!")
display(ipd.Audio(strong_signal, rate=SAMPLE_RATE_WSJ0))

enc_basis trainable
dec_basis trainable
relu
None
None
2
Choose layer norm gLN 64 0 1e-12
mask_nonlinear sigmoid
{'in_channels': 1, 'n_basis': 64, 'kernel_size': 16, 'stride': 8, 'enc_basis': 'trainable', 'dec_basis': 'trainable', 'enc_nonlinear': 'relu', 'window_fn': None, 'enc_onesided': None, 'enc_return_complex': None, 'sep_hidden_channels': 256, 'sep_bottleneck_channels': 128, 'sep_skip_channels': 128, 'sep_kernel_size': 3, 'sep_num_blocks': 3, 'sep_num_layers': 6, 'dilated': 1, 'separable': 1, 'causal': 0, 'sep_nonlinear': 'prelu', 'sep_norm': 1, 'mask_nonlinear': 'sigmoid', 'n_sources': 2, 'eps': 1e-12}
torch.Size([1, 53841])
Input size torch.Size([1, 1, 53841])
torch.Size([1, 2, 53841])
Calling pick strongest signal
In pick
First one second
tensor(401.0006) tensor(166.7026)
Entire signal
tensor(683.2902) tensor(259.5116)


the strong signal!


In [ ]:

m = torch.nn.GroupNorm(1,3)
x = torch.randn(1,3,)
#print(torch.mean(x,2, keepdim=True))
#print(torch.var(x,2, keepdim=True))
print(torch.mean(x))
x = x-torch.mean(x)
x= x/torch.sqrt(torch.var(x)+1e-5)
print(x)
y=m(x)
print(m.weight)
print(y/x)

In [ ]:


model = DPRNNTasNet.build_from_pretrained(task="wsj0-mix", sample_rate=SAMPLE_RATE_WSJ0, n_sources=n_sources)
model.eval()

input = mixture.unsqueeze(dim=0)
with torch.no_grad():
    output = model(input)

output = output.squeeze(dim=0)
estimated = torch.split(output, [1]*n_sources, dim=0)

for idx in range(n_sources):
    display(ipd.Audio(estimated[idx], rate=SAMPLE_RATE_WSJ0))

In [ ]:
model = DPTNet.build_from_pretrained(task="wsj0-mix", sample_rate=SAMPLE_RATE_WSJ0, n_sources=n_sources)
model.eval()

input = mixture.unsqueeze(dim=0)
with torch.no_grad():
    output = model(input)

output = output.squeeze(dim=0)
estimated = torch.split(output, [1]*n_sources, dim=0)

for idx in range(n_sources):
    display(ipd.Audio(estimated[idx], rate=SAMPLE_RATE_WSJ0))

In [ ]:
model = SepFormer.build_from_pretrained(task="wsj0-mix", sample_rate=SAMPLE_RATE_WSJ0, n_sources=n_sources)
model.eval()

input = mixture.unsqueeze(dim=0)
with torch.no_grad():
    output = model(input)

output = output.squeeze(dim=0)
estimated = torch.split(output, [1]*n_sources, dim=0)

for idx in range(n_sources):
    display(ipd.Audio(estimated[idx], rate=SAMPLE_RATE_WSJ0))

## 3 speakers

In [ ]:
n_sources = 3
mixture = waveform_aew + waveform_axb + waveform_bdl
display(ipd.Audio(mixture, rate=SAMPLE_RATE_WSJ0))

In [ ]:
model = DeepClustering.build_from_pretrained(task="wsj0-mix", sample_rate=SAMPLE_RATE_WSJ0, n_sources=n_sources)
wrapper_model = DeepClustering.TimeDomainWrapper(model, n_fft=model.n_fft, hop_length=model.hop_length, window_fn=model.window_fn)
wrapper_model.eval()

input = mixture.unsqueeze(dim=0)
with torch.no_grad():
    output = wrapper_model(input, threshold=model.threshold, n_sources=n_sources, iter_clustering=None)

output = output.squeeze(dim=0)
estimated = torch.split(output, [1]*n_sources, dim=0)

for idx in range(n_sources):
    display(ipd.Audio(estimated[idx], rate=SAMPLE_RATE_WSJ0))

In [ ]:
model = DANet.build_from_pretrained(task="wsj0-mix", sample_rate=SAMPLE_RATE_WSJ0, n_sources=n_sources)
wrapper_model = DANet.TimeDomainWrapper(model, n_fft=model.n_fft, hop_length=model.hop_length, window_fn=model.window_fn)
wrapper_model.eval()

input = mixture.unsqueeze(dim=0)
with torch.no_grad():
    output = wrapper_model(input, threshold=model.threshold, n_sources=n_sources, iter_clustering=None)

output = output.squeeze(dim=0)
estimated = torch.split(output, [1]*n_sources, dim=0)

for idx in range(n_sources):
    display(ipd.Audio(estimated[idx], rate=SAMPLE_RATE_WSJ0))

In [ ]:
model = FixedAttractorDANet.build_from_pretrained(task="wsj0-mix", sample_rate=SAMPLE_RATE_WSJ0, n_sources=n_sources)
wrapper_model = FixedAttractorDANet.TimeDomainWrapper(model, n_fft=model.n_fft, hop_length=model.hop_length, window_fn=model.window_fn)
wrapper_model.eval()

input = mixture.unsqueeze(dim=0)
with torch.no_grad():
    output = wrapper_model(input)

output = output.squeeze(dim=0)
estimated = torch.split(output, [1]*n_sources, dim=0)

for idx in range(n_sources):
    display(ipd.Audio(estimated[idx], rate=SAMPLE_RATE_WSJ0))

In [ ]:
model = ADANet.build_from_pretrained(task="wsj0-mix", sample_rate=SAMPLE_RATE_WSJ0, n_sources=n_sources)
wrapper_model = ADANet.TimeDomainWrapper(model, n_fft=model.n_fft, hop_length=model.hop_length, window_fn=model.window_fn)
wrapper_model.eval()

input = mixture.unsqueeze(dim=0)
with torch.no_grad():
    output = wrapper_model(input, threshold=40, n_sources=n_sources)

output = output.squeeze(dim=0)
estimated = torch.split(output, [1]*n_sources, dim=0)

for idx in range(n_sources):
    display(ipd.Audio(estimated[idx], rate=SAMPLE_RATE_WSJ0))

In [ ]:
model = LSTMTasNet.build_from_pretrained(task="wsj0-mix", sample_rate=SAMPLE_RATE_WSJ0, n_sources=n_sources)
model.eval()

input = mixture.unsqueeze(dim=0)
with torch.no_grad():
    output = model(input)

output = output.squeeze(dim=0)
estimated = torch.split(output, [1]*n_sources, dim=0)

for idx in range(n_sources):
    display(ipd.Audio(estimated[idx], rate=SAMPLE_RATE_WSJ0))

In [ ]:
model = ConvTasNet.build_from_pretrained(task="wsj0-mix", sample_rate=SAMPLE_RATE_WSJ0, n_sources=n_sources)
model.eval()

input = mixture.unsqueeze(dim=0)
with torch.no_grad():
    output = model(input)

output = output.squeeze(dim=0)
estimated = torch.split(output, [1]*n_sources, dim=0)

for idx in range(n_sources):
    display(ipd.Audio(estimated[idx], rate=SAMPLE_RATE_WSJ0))

In [ ]:
model = DPRNNTasNet.build_from_pretrained(task="wsj0-mix", sample_rate=SAMPLE_RATE_WSJ0, n_sources=n_sources)
model.eval()

input = mixture.unsqueeze(dim=0)
with torch.no_grad():
    output = model(input)

output = output.squeeze(dim=0)
estimated = torch.split(output, [1]*n_sources, dim=0)

for idx in range(n_sources):
    display(ipd.Audio(estimated[idx], rate=SAMPLE_RATE_WSJ0))

In [ ]:
model = DPTNet.build_from_pretrained(task="wsj0-mix", sample_rate=SAMPLE_RATE_WSJ0, n_sources=n_sources)
model.eval()

input = mixture.unsqueeze(dim=0)
with torch.no_grad():
    output = model(input)

output = output.squeeze(dim=0)
estimated = torch.split(output, [1]*n_sources, dim=0)

for idx in range(n_sources):
    display(ipd.Audio(estimated[idx], rate=SAMPLE_RATE_WSJ0))

In [ ]:
model = SepFormer.build_from_pretrained(task="wsj0-mix", sample_rate=SAMPLE_RATE_WSJ0, n_sources=n_sources)
model.eval()

input = mixture.unsqueeze(dim=0)
with torch.no_grad():
    output = model(input)

output = output.squeeze(dim=0)
estimated = torch.split(output, [1]*n_sources, dim=0)

for idx in range(n_sources):
    display(ipd.Audio(estimated[idx], rate=SAMPLE_RATE_WSJ0))